In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install fastapi uvicorn pyngrok transformers accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.6 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from huggingface_hub import login
login()

In [4]:
!pip -q install "transformers>=4.43" "accelerate>=0.33" "bitsandbytes>=0.43" "autoawq>=0.2.7" torch --extra-index-url https://download.pytorch.org/whl/cu121

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 28.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
bigframes 2.8.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.8.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch, os, shutil, pathlib

base_id = "mistralai/Mistral-Nemo-Instruct-2407"
tok = AutoTokenizer.from_pretrained(base_id, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
bnb_cfg = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained(base_id, quantization_config=bnb_cfg, device_map="auto")

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

2025-09-07 17:19:15.047749: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757265555.219838      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757265555.268345      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

...

model-00002-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
save_dir = "mistral_nemo_instruct_2407_4bit_bnb"
try:
    model.save_pretrained(save_dir, safe_serialization=True)
    tok.save_pretrained(save_dir)
    print("Saved quantized model to:", save_dir)
except Exception as e:
    print("Save 4-bit not supported in this env:", e)

Saved quantized model to: mistral_nemo_instruct_2407_4bit_bnb


In [8]:
import os, time, math, shutil
from pathlib import Path
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

save_dir = "/kaggle/working/mistral_nemo_instruct_2407_4bit_bnb"

def load_model_and_tokenizer(path_or_id):
    bnb_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
    tok = AutoTokenizer.from_pretrained(path_or_id, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        path_or_id,
        quantization_config=bnb_cfg,
        device_map="auto",
        torch_dtype=torch.bfloat16,
    )
    return tok, model

In [9]:
if Path(save_dir).exists():
    print(f"Loading local 4-bit checkpoint: {save_dir}")
    tok, model = load_model_and_tokenizer(save_dir)
else:
    print("Local 4-bit checkpoint not found (saving likely not supported in this env).")
    print(f"Falling back to: {fallback_hub_id}")
    tok, model = load_model_and_tokenizer(fallback_hub_id)

def generate(prompt, max_new_tokens=128, temperature=0.7, top_p=0.95):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    t0 = time.time()
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        pad_token_id=tok.eos_token_id,
    )
    t1 = time.time()
    text = tok.decode(out[0], skip_special_tokens=True)
    print(f"\n--- Generation ({t1 - t0:.2f}s) ---\n{text}\n")
    return text

Loading local 4-bit checkpoint: /kaggle/working/mistral_nemo_instruct_2407_4bit_bnb


/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
_ = generate("Explain model quantization to students in two sentences.")
_ = generate("Give three bullet points comparing fp16 vs int8 vs 4-bit quantization.")


--- Generation (9.73s) ---
Explain model quantization to students in two sentences.Quantization is a technique used in machine learning model optimization that reduces the precision of weights and activations in a model, typically from float32 to int8 or float16, without significantly sacrificing model accuracy. This process not only reduces the model's memory footprint but also speeds up inference time, making it more efficient for deployment on resource-limited devices.


--- Generation (14.52s) ---
Give three bullet points comparing fp16 vs int8 vs 4-bit quantization.  
- **fp16**:
  - **Precision**: fp16 has half the number of bits compared to fp32, offering reduced precision but retaining some flexibility for complex operations.
  - **Memory Efficiency**: fp16 saves half the memory compared to fp32, but it's less efficient than int8 and 4-bit quantization.
  - **Speed**: fp16 operations are typically faster than fp32 on hardware supporting it, like Nvidia GPUs.

- **int8**:
  - *

In [11]:
NGROK_TOKEN = "32NdJS7skdlVn1em50DJ77sUiju_AFPKSCuPnJcxvND2XVY3"
API_KEY = "secret123"

In [14]:
from fastapi import FastAPI, Request, HTTPException
import uvicorn, threading, time, socket
from pyngrok import ngrok, conf

app = FastAPI()

@app.post("/generate")
async def gen(req: Request):
    if req.headers.get("authorization") != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")
    data = await req.json()
    return {
        "response": generate(
            data.get("prompt", ""),
            data.get("max_length", 300)
        )
    }

In [15]:
def free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = free_port()
conf.get_default().auth_token = NGROK_TOKEN
public_url = ngrok.connect(port).public_url
print("Your public URL:", public_url)

def run(): uvicorn.run(app, host="0.0.0.0", port=port)
threading.Thread(target=run, daemon=True).start()
time.sleep(1)

Your public URL: https://017f01129768.ngrok-free.app


INFO:     Started server process [36]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:60737 (Press CTRL+C to quit)



--- Generation (31.09s) ---
who is CR7? Cristiano Ronaldo dos Santos Aveiro Ronaldo Jr., GOIH, commonly known as Cristiano Ronaldo, is a Portuguese professional footballer who plays as a forward for Saudi Arabian club Al Nassr and captains the Portugal national team.He is often considered the best player in the world and has repeatedly been ranked first or second best player in the world in various publications.

Ronaldo holds several records and accolades throughout his career. Here are some highlights:

1. **Club Career:**
   - He played for Manchester United from 2003 to 2009, winning three Premier League titles and the UEFA Champions League in 2008.
   - From 2009 to 2018, he played for Real Madrid, winning two La Liga titles, two Copa del Rey titles, and four UEFA Champions League titles.
   - Since 2018, he has played for Juventus, winning two Serie A titles and the Coppa Italia in his first season.
   - Currently, he plays for Al Nassr in Saudi Arabia.

2. **International Caree